In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, LimitOrder, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

# Imports 
from utils import Vars, prepare_to_sow, place_orders, quick_pf, cancel_all_api_orders, create_target_opts, get_pickle
from loguru import logger

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

# Correct `infinity ROM` mistake!!!

In [ ]:
import numpy as np
import asyncio 
from utils import get_order_pf, get_prec

df = get_pickle(DATAPATH / 'df_sow.pkl')

condition = df.rom == np.inf
df[condition][OPT_COLS]

df_wrong_rom = df[df.rom == np.inf]

In [ ]:

order, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
wrong_rom = set(df_pf.conId.to_list()).intersection(set(df.conId.to_list()))

df_wronged = df_pf[df_pf.conId.isin(wrong_rom)]

In [ ]:
# get price of df_wronged
from utils import get_opt_price_ivs, qualify_conIds, clean_ib_util_df

contracts = asyncio.run(qualify_conIds(PORT, df_wronged.conId.to_list(), f"Qualifying {MARKET} which are WRONGED"))

with IB().connect(port=PORT) as ib:
    df_prices = asyncio.run(get_opt_price_ivs(port=PORT, qualified_opts=contracts, HIGHEST=False))

In [ ]:
# integrate the prices of wronged df into portfolio
cols = list(df_prices.columns[:7]) + ['contract', 'bid', 'ask', 'last', 'close','optPrice']
df_p = df_prices[cols]

In [ ]:
p_cols = df_p.columns.difference(df_pf.columns)
df_pf1 = df_pf.set_index('conId').join(df_p.set_index('conId')[p_cols])

In [ ]:
# extract conIds without contracts and update
condition = df_pf1.contract.isnull()
conIds = df_pf1[condition].index
contracts = asyncio.run(qualify_conIds(PORT, list(conIds)))
df_good = clean_ib_util_df(contracts).set_index('conId')[['multiplier', 'contract']]
df_pf1.update(df_good)

In [ ]:
# integrate the prices for missing contracts
with IB().connect(port=PORT) as ib:
    df_good_price = asyncio.run(get_opt_price_ivs(port=PORT, qualified_opts=contracts, HIGHEST=False))
df_good_price = df_good_price.set_index('conId').optPrice

df_pf1.update(df_good_price)

In [ ]:
cols1 = [c for c in cols + ['avgCost'] if c not in ['conId', 'contract']]
df_pf2 = df_pf1.loc[df_pf1.optPrice.div(df_pf1.avgCost).sort_values(ascending=False).index]

In [ ]:
expPrice = np.where(df_pf2.optPrice > df_pf2.avgCost, (df_pf2.optPrice + df_pf2.avgCost)/2, df_pf2.optPrice)
series_expPrice = df_pf2.assign(expPrice = expPrice).expPrice.apply(lambda x: get_prec(x - _vars.PREC, _vars.PREC))

In [ ]:
df_pf3 = df_pf2.join(series_expPrice)

In [ ]:
cos = list(zip(df_pf3.contract, [LimitOrder('BUY', qty, price) for qty, price in zip(abs(df_pf3.position), df_pf3.expPrice)]))

In [ ]:
df_pf3[cols1 + ['expPrice']]